In [5]:
from pandas import read_csv
from os import listdir
import cv2
import numpy as np

In [6]:
## converts the percent positions into integer positions
# def unconvert(width, height, x, y, w, h):
#     xmin = int(x*width)
#     xmax = int(y*width)
#     ymin = int(w*height)
#     ymax = int(h*height)
#     return (abs(xmin), abs(xmax), abs(ymin), abs(ymax))

def unconvert(width, height, x, y, w, h):
    xmin = x*width
    xmax = y*width
    ymin = w*height
    ymax = h*height
    return xmin, xmax, ymin, ymax


def get_max_iou(pred_boxes, gt_box):
    if pred_boxes.shape[0] > 0:
        ixmin = np.maximum(pred_boxes[:, 0], gt_box[0])
        ixmax = np.minimum(pred_boxes[:, 2], gt_box[2])
        iymin = np.maximum(pred_boxes[:, 1], gt_box[1])
        iymax = np.minimum(pred_boxes[:, 3], gt_box[3])
        
        ###########
        iw = np.maximum(ixmax - ixmin + 1., 0.)
        ih = np.maximum(iymax - iymin + 1., 0.)

        inters = iw * ih
    # calculates Union 
        uni = ((pred_boxes[:, 2] - pred_boxes[:, 0] + 1.) * (pred_boxes[:, 3] - pred_boxes[:, 1] + 1.) + (gt_box[2] - gt_box[0] + 1.) * (gt_box[3] - gt_box[1] + 1.) - inters)
    # calculates Max of two areas
        ###########
#         iw = np.maximum(ixmax - ixmin + 1., 0.)
#         ih = np.maximum(iymax - iymin + 1., 0.)

#         inters = abs(iw * ih)
#         # calculates Union 
#         uni = ((pred_boxes[:, 2] - pred_boxes[:, 0] + 1.) * (pred_boxes[:, 3] - pred_boxes[:, 1] + 1.) + (gt_box[2] - gt_box[0] + 1.) * (gt_box[3] - gt_box[1] + 1.) - inters)
        # calculates Max of two areas
#         uni = np.maximum((pred_boxes[:, 2] - pred_boxes[:, 0] + 1.) * (pred_boxes[:, 3] - pred_boxes[:, 1] + 1.), (gt_box[2] - gt_box[0] + 1.) * (gt_box[3] - gt_box[1] + 1.))
        iou = inters / uni
        iou = iou.tolist()
        iou_max = np.max(iou)
        nmax = np.argmax(iou)
        return iou, iou_max, nmax

In [7]:
def read_all_images(data_dir, annot_dir):
    df = read_csv(annot_dir)
    data_folders = listdir(data_dir)
    img_list = df.ImageID.tolist()
    image_list = []                                      
    for folder in data_folders:
        if folder.endswith('.csv') or folder.endswith('.ipynb'):
            continue
        image_folder_dir = data_dir + '/' + folder
        images = listdir(image_folder_dir)
        for image in images:
            if image.endswith('.jpg') and image[:-4] in img_list:
                image_path = image_folder_dir + '/' + image
                image_ = cv2.imread(image_path)
                height, width, channels = image_.shape
                image_data = df.loc[(df['ImageID'] == image[0:-4])]
                ind = [index for index, row in image_data.iterrows()]
                bx = [unconvert(width, height, row['XMin'], row['XMax'], row['YMin'], row['YMax']) for index, row in image_data.iterrows()]
                bx = np.array(bx)
                counter = 0
                for index, row in image_data.iterrows():
                    if counter < len(bx):
                        iou, iou_max, nmax = get_max_iou(bx, bx[counter])
                        image_data['ImageID']
                        to_remove = []
                        for i in range(len(iou)):
                            if i is not counter and iou[i] > .7:
                                to_remove.append(i)  
                        for i in to_remove[::-1]:
                            df = df.drop([ind[i]])
                            ind = ind[:i] + ind[i+1 :]
                            bx = bx.tolist()
                            bx = bx[:i] + bx[i+1:]
                            bx = np.array(bx)
                        counter += 1
                    else:
                        break
    df.to_csv('E:/CEMTREX/testing/exported_csv.csv',index=False)

In [8]:
annot_dir = 'O:/daaaCEMTREX/openImgCSv/v6/labels/validation-annotations-bbox.csv'
data_dir = 'O:/daaaCEMTREX/openImgCSv/v6/open_data/val_present/'
read_all_images(data_dir, annot_dir)

c:\users\_kale\pycharmprojects\first\venv\lib\site-packages\ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in true_divide


In [9]:
import pandas as pd
import os as os
import numpy as np

exp_annot_dir = 'E:/CEMTREX/testing/exported_csv.csv'
df = pd.read_csv(exp_annot_dir)
df.head
data_folders = os.listdir(data_dir)
i = 0
for folder in data_folders:
    # import pdb; pdb.set_trace()
    if folder.endswith('.csv') or folder.endswith('.ipynb'):
        continue
    image_folder_dir = data_dir + '/' + folder
    images = os.listdir(image_folder_dir)
    image_list = [i[:-4] for i in images]
    new_df = df[df['ImageID'].isin(image_list)]
    class_df = pd.read_csv("O:/downloads/required_classes.csv")
    class_list = class_df.ClassID.tolist()
    # print(class_list)
    n1 = new_df[(new_df["LabelName"].isin(class_list))]
    
#     n2 = n1.replace(('/m/01bl7v','/m/03bt1vf','/m/04yx4','/m/05r655',),'/m/01g317') #renaming man,women,boy,girl to person
#     n3 = n2.replace(('/m/01lcw4','','',''),'')
    n1.to_csv(image_folder_dir+'/'+folder+".csv")
    i = i+1
    if i ==2:break
        